In [87]:
import sys
import os
# Add the SDK directory to the Python path
# This assumes your notebook is in services/engine-cpp/notebooks/
# and the SDK is in sdk/python/
sdk_path = os.path.abspath("../../../sdk/python")
root = os.path.abspath("../../../")
print(f"project root path = {root}")
print(f"sdk path = {sdk_path}")

if sdk_path not in sys.path:
    sys.path.append(sdk_path)

from client import RoutingClient

# Initialize the client (defaults to http://localhost:8082)
client = RoutingClient("http://localhost:8082")

# Check health
print(client.health())

# Run a route (example coordinates for Somerset)
response = client.route(
    dataset="somerset",
    start_lat=37.08093, start_lng=-84.61447,
    end_lat=37.08, end_lng=-84.60
)

if response.success:
    print(f"Path distance: {response.distance_meters}m")
    print(f"Path edges: {response.path}")
else:
    print(f"Error: {response.error}")

project root path = /home/kaveh/projects/h3-routing-platform
sdk path = /home/kaveh/projects/h3-routing-platform/sdk/python
Loaded local config from /home/kaveh/projects/h3-routing-platform/services/api-gateway/config/datasets.yaml with 4 datasets
{'datasets_loaded': [], 'status': 'healthy'}
Error: Dataset 'somerset' not loaded


In [5]:
#client.unload_dataset(name="somerset")

True

In [58]:
import subprocess

def get_memory_usage_mb(extra=""):
    try:
        # Run ps command to get RSS (Resident Set Size) in KB
        output = subprocess.check_output(
            f"ps -C routing_server{extra} -o rss=", 
            shell=True
        ).decode().strip()
        
        if not output: return 0.0
        
        # Convert KB to MB (RSS is in Kilobytes)
        rss_kb = int(output.split()[0])
        return rss_kb / 1024.0
    except Exception as e:
        print(f"Error reading memory: {e}")
        return 0.0

# Usage
server_memo = get_memory_usage_mb()
print(f"Current Memory Usage: {server_memo:.2f} MB")

Error reading memory: Command 'ps -C routing_server -o rss=' returned non-zero exit status 1.
Current Memory Usage: 0.00 MB


In [ ]:
# old server : 5892
# old server:  3000

# crs : 6.09
# crs : 4957.66 MB
# crs_unloaded : 1865.18 MB

#old_server : 27
#old_server : 5896.60 MB
#old_server_unloaded : 3005.79 MB


In [90]:
client.load_dataset(name="metro_vancouver", db_path=f"{root}/data/All_Vancouver.db")

True

In [82]:
server_metro_vancouver = get_memory_usage_mb("_csr")
print(f"Current Memory Usage: {server_metro_vancouver:.2f} MB")

Current Memory Usage: 1865.18 MB


In [107]:
server_metro_vancouver = get_memory_usage_mb()
print(f"Current Memory Usage: {server_metro_vancouver:.2f} MB")

Current Memory Usage: 3005.79 MB


In [104]:
client.unload_dataset(name="metro_vancouver")

True

In [94]:
print(client.health())

{'datasets_loaded': ['metro_vancouver'], 'status': 'healthy'}


In [17]:
# Route test cases (source_lat, source_lon, target_lat, target_lon)
TEST_ROUTES = [
    (49.28, -123.12, 49.25, -123.10),  # Downtown to Fairview
    (49.23, -123.00, 49.28, -122.85),  # South to North Burnaby
    (49.20, -122.90, 49.30, -123.05),  # South Burnaby to North Van
    (49.25, -122.80, 49.22, -123.15),  # Coquitlam to UBC
    (49.15, -122.95, 49.26, -122.75),  # Surrey to Port Moody
]

In [ ]:
# Route test cases (source_lat, source_lon, target_lat, target_lon)
TEST_ROUTES = [
    (49.28, -123.12, 49.25, -123.10),  # Downtown to Fairview
    (49.23, -123.00, 49.28, -122.85),  # South to North Burnaby
    (49.20, -122.90, 49.30, -123.05),  # South Burnaby to North Van
    (49.25, -122.80, 49.22, -123.15),  # Coquitlam to UBC
    (49.15, -122.95, 49.26, -122.75),  # Surrey to Port Moody
]

def test_routes(server_url="http://localhost:8082"):
    """Run test routes and return timing stats."""
    times = []
    
    for i, (s_lat, s_lon, t_lat, t_lon) in enumerate(TEST_ROUTES):
        try:
            resp = requests.post(
                f"{server_url}/route",
                json={
                    "dataset": "metro_vancouver",
                    "start_lat": s_lat,
                    "start_lng": s_lon,
                    "end_lat": t_lat,
                    "end_lng": t_lon,
                    "mode": "knn"
                },
                timeout=30
            )
            data = resp.json()
            runtime = data.get("runtime_ms", 0)
            distance = data.get("distance", 0)
            print(f"  Route {i+1}: {runtime:.1f} ms (distance: {distance:.1f})")
            times.append(runtime)
        except Exception as e:
            print(f"  Route {i+1}: ERROR - {e}")
    
    if times:
        avg = sum(times) / len(times)
        print(f"  Average: {avg:.1f} ms")
        return avg
    return None



In [100]:
response_list = list()
for item in TEST_ROUTES:
    response = client.route( 
            dataset="metro_vancouver",
            start_lat = item[0],
            start_lng = item[1],
            end_lat = item[2],
            end_lng = item[3],
            mode = "knn",
            num_candidates = 1
        ) 
    response_list.append(response)

In [ ]:
df = 

In [29]:
import pandas as pd

In [101]:
df = pd.DataFrame(response_list)

In [102]:
df.head()

,success,distance,distance_meters,runtime_ms,path,geojson,error
0,True,121.335811,5055.805,252.609868,"[153359, 153361, 116637, 116639, 116644, 24960...",{'geometry': {'coordinates': [[-123.1199766328...,None
1,True,278.618574,16243.616,388.485974,"[278661, 278676, 278678, 259579, 154693, 15468...",{'geometry': {'coordinates': [[-123.0000204661...,None
2,True,547.486564,27185.859,518.158535,"[106269, 106307, 106268, 100420, 157326, 98919...",{'geometry': {'coordinates': [[-122.8992401611...,None
3,True,519.225435,33178.600,1184.361445,"[203237, 77009, 203236, 72641, 82376, 82233, 8...",{'geometry': {'coordinates': [[-122.7999336724...,None
4,True,416.480919,26622.947,605.082001,"[273485, 273477, 273470, 284443, 195399, 24301...",{'geometry': {'coordinates': [[-122.9501926321...,None


In [98]:
df_CSR = df.copy()

In [99]:
df_CSR

,success,distance,distance_meters,runtime_ms,path,geojson,error
0,True,124.466164,5174.956,350.671241,"[153359, 153361, 116637, 116639, 116644, 24960...",{'geometry': {'coordinates': [[-123.1199766328...,None
1,True,281.120735,16202.901,436.308460,"[278666, 278661, 278676, 278678, 259579, 15469...",{'geometry': {'coordinates': [[-123.0000610351...,None
2,True,501.569404,26071.201,624.871470,"[106269, 106307, 106268, 100420, 157326, 98919...",{'geometry': {'coordinates': [[-122.8992401611...,None
3,True,523.130668,33295.757,1455.569550,"[203237, 77009, 203236, 72641, 82376, 82233, 8...",{'geometry': {'coordinates': [[-122.7999336724...,None
4,True,443.603555,27829.158,704.559754,"[194006, 194005, 193977, 193980, 193983, 19398...",{'geometry': {'coordinates': [[-122.9502182006...,None
